In [1]:
%pip install pytorch-pretrained-bert

Note: you may need to restart the kernel to use updated packages.


In [2]:
# !pip install --upgrade tensorflow-addons

In [3]:
# !pip install 'keras<3.0.0' mediapipe-model-maker

In [4]:
%pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
# !pip install --upgrade keras

In [6]:
# import keras
# from keras.models import load_model

In [7]:
# !pip install tensorflow_addons

In [8]:
# pip install tensorflow -

In [9]:
%pip install 'keras<3.0.0' mediapipe-model-maker

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


In [136]:
import torch
import torch.nn as nn
import torchvision.utils as vutils
import torchvision.transforms as transforms
import os
import sys
input_dir = r'D:\clg files\RecSys\py_files'
sys.path.append(input_dir)

from data_util import AttDesDataset
from utils import weights_init

from dcgan_model import Generator, Discriminator
import time
import imageio
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [137]:
# Setting device to cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using Device", device)

Using Device cpu


In [138]:
# directory to store output images
output_save_path = './generated_images/'
os.makedirs(output_save_path, exist_ok=True)

In [139]:
# directory to store trained models
model_save_path = './saved_models/'
os.makedirs(model_save_path, exist_ok=True)

In [140]:
from torch.utils.data import Dataset

In [141]:
from torch.nn.utils.rnn import pad_sequence

class CustomDataset(Dataset):
    def __init__(self, data):
        self.images = data['images']
        self.captions = data['captions']

        # Build a vocabulary from the captions
        self.vocab = self.build_vocab(self.captions)

    def build_vocab(self, captions):
        vocab = set()
        for caption in captions:
            vocab.update(list(caption))  # tokenize by character
        vocab = sorted(vocab)
        vocab.append('<unk>')  # add unknown token
        vocab.append('<pad>')  # add padding token
        vocab.append('<bos>')  # add beginning of sentence token
        vocab.append('<eos>')  # add end of sentence token
        word2idx = {word: idx for idx, word in enumerate(vocab)}
        return word2idx

    def __getitem__(self, index):
        image = self.images[index]
        caption = self.captions[index]

        # Tokenize the caption and convert it to tensor
        tokenized_caption = list(caption)  # tokenize by character
        caption_tensor = torch.tensor([self.vocab.get(token, self.vocab['<unk>']) for token in tokenized_caption])

        return image, caption_tensor

    def __len__(self):
        return len(self.images)

    def collate_fn(self, data):
        # Sort the data in descending order of caption length
        data.sort(key=lambda x: len(x[1]), reverse=True)

        images, captions = zip(*data)

        # Pad the captions
        lengths = [len(cap) for cap in captions]
        targets = pad_sequence([torch.tensor(cap) for cap in captions], batch_first=True, padding_value=self.vocab['<pad>'])

        return images, targets, lengths

In [142]:
# data_root = r'/kaggle/input/textfor/data_for_test2.csv'
# split_root = ''
# dataset_name = 'Furniture'
# normalize = transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
# batch_size = 1 #128
# dataset = AttDesDataset(data_root, dataset_name, transform=transforms.Compose([
#                                             transforms.Resize((448,448)),
#                                             transforms.RandomHorizontalFlip(),
#                                             transforms.ToTensor(),
#                                             normalize,
#                                         ]))

# print(dataset.images)
# print(dataset.des_list)

# training_dict = {
#     'images': dataset.images,
#     'captions': dataset.des
# }
# train_loader = DataLoader(training_dict, batch_size=batch_size,shuffle=True,num_workers=8)
# print("No of batches: ",len(train_loader))

from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

data_root = r'data_files\smoll.csv'
split_root = ''
dataset_name = 'Furniture'
normalize = transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
batch_size = 64 #128
dataset = AttDesDataset(data_root, dataset_name, transform=transforms.Compose([
                                            transforms.Resize((448,448)),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            normalize,
                                        ]))

temp_dataset = {'images': dataset.images, 'captions': dataset.des_list}
custom_dataset = CustomDataset(temp_dataset)
print(len(custom_dataset))
# print(len(custom_dataset['captions']))
# Assuming your dataset has 1000 samples
num_samples = len(custom_dataset)
indices = list(range(num_samples))
split = int(np.floor(0.8 * num_samples))  # 80-20 train-validation split

# Randomly shuffle the indices
np.random.shuffle(indices)

# Split the indices into train and validation sets
train_indices, val_indices = indices[:split], indices[split:]

# Define samplers for train and validation sets
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Define DataLoader for train and validation sets using samplers
# train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=8)
train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=custom_dataset.collate_fn)
val_loader = DataLoader(custom_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=0, collate_fn=custom_dataset.collate_fn)

# Check the number of batches in train and validation loaders
print("No of batches in train loader: ", len(train_loader))
print("No of batches in validation loader: ", len(val_loader))



1860
No of batches in train loader:  30
No of batches in validation loader:  6


In [143]:
for batch_idx,batch in enumerate(train_loader):
    print(batch_idx)
    # print(batch)
    print(type(batch[0][0]))
    print(len(batch[0]))
    print(type(batch[1]))
    if(batch_idx == 1):
        break

0
<class 'torch.Tensor'>
64
<class 'torch.Tensor'>
1
<class 'torch.Tensor'>
64
<class 'torch.Tensor'>


In [144]:
# setting up parameters
noise_dim = 100
embed_dim = 256
embed_out_dim = 128
batch_size = 64 #128
real_label = 1.
fake_label = 0.
learning_rate = 0.0002
l1_coef = 50
l2_coef = 100

num_epochs = 1
log_interval = 18 #43

In [145]:
# loss functions
criterion = nn.BCELoss()
l2_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

In [146]:
# lists to store losses
D_losses = []
G_losses = []

In [147]:
# initializing generator
generator = Generator(channels=3, embed_dim=embed_dim, noise_dim=noise_dim, embed_out_dim=embed_out_dim).to(device)
generator.apply(weights_init)

Generator(
  (text_embedding): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (model): Sequential(
    (0): ConvTranspose2d(228, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=Fal

In [148]:
# initializing discriminator
discriminator = Discriminator(channels=3, embed_dim=embed_dim, embed_out_dim=embed_out_dim).to(device)
discriminator.apply(weights_init)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (text_embedding): Embedding(
    (text_embedding): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, mom

In [149]:
# setting up Adam optimizer for Generator and Discriminator
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [150]:
# training loop
import torch.nn.functional as F

# iterating over number of epochs
from datetime import date

start_time = time.time()
for epoch in range(num_epochs):
    
    batch_time = time.time()
    print('Epoch: {}'.format(epoch+1))
    #iterating over each batch
    for batch_idx,batch in enumerate(train_loader):   

        print(f'epoch: {epoch+1}, batch: {batch_idx+1}/{len(train_loader)}')
        
        # reading the data into variables and moving them to device
        images = batch[0][0].to(device)
        # print('hi after images')
        wrong_images = batch[0][0].to(device)
        # print('hi after wrong images')
        embeddings = batch[1].to(device).float()

        fixed_size = 256 
        padding = fixed_size - embeddings.size(1)

        # Pad the tensor
        if padding > 0:
            embeddings = F.pad(embeddings, (0, padding))
        elif padding < 0:
            embeddings = embeddings[:, :fixed_size]

        batch_size = images.size(0)
        print(f'embeddings size: {embeddings.size()}')
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        print('training the descriminator')
        # Clear gradients for the discriminator
        optimizer_D.zero_grad()
        
        # Generate random noise
        noise_dim = 256  
        batch_size = images.size(0)
        noise = torch.randn(batch_size, noise_dim).to(device)
        noise = noise.view(noise.shape[0], noise.shape[1], 1, 1)

        # Reshape the embeddings tensor to match the noise tensor
        embeddings = embeddings.view(embeddings.shape[0], embeddings.shape[1], 1, 1)

        # Generate fake image batch with the generator
        print(noise.shape,embeddings.shape)
        fake_images = generator(noise, embeddings)
        print('hi after fake images')
        # Forward pass real batch and calculate loss
        real_out, real_act = discriminator(images, embeddings)
        d_loss_real = criterion(real_out, torch.full_like(real_out, real_label, device=device))
        
        # Forward pass wrong batch and calculate loss
        wrong_out, wrong_act = discriminator(wrong_images, embeddings)
        d_loss_wrong = criterion(wrong_out, torch.full_like(wrong_out, fake_label, device=device))
        
        # Forward pass fake batch and calculate loss
        fake_out, fake_act = discriminator(fake_images.detach(), embeddings)
        d_loss_fake = criterion(fake_out, torch.full_like(fake_out, fake_label, device=device))
        
        # Compute total discriminator loss
        d_loss = d_loss_real + d_loss_wrong + d_loss_fake
        
        # Backpropagate the gradients
        d_loss.backward()
        
        # Update the discriminator
        optimizer_D.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #
        print('Training the generator')
        # Clear gradients for the generator
        optimizer_G.zero_grad()
       
        # Generate new fake images using Generator
        fake_images = generator(noise, embeddings)
        
        # Get discriminator output for the new fake images
        out_fake, act_fake = discriminator(fake_images, embeddings)
        
        # Get discriminator output for the real images
        out_real, act_real = discriminator(images, embeddings)
        
        # Calculate losses
        g_bce = criterion(out_fake, torch.full_like(out_fake, real_label, device=device)) 
        g_l1 = l1_coef * l1_loss(fake_images, images)
        g_l2 = l2_coef * l2_loss(torch.mean(act_fake, 0), torch.mean(act_real, 0).detach())
        
        # Compute total generator loss
        g_loss = g_bce + g_l1 + g_l2
        
        # Backpropagate the gradients
        g_loss.backward()
        
        # Update the generator
        optimizer_G.step()
        
        # adding loss to the list
        D_losses.append(d_loss.item())
        G_losses.append(g_loss.item())
        
        # progress based on log_interval
        if (batch_idx+1) % log_interval == 0 and batch_idx > 0:
            print('Epoch {} [{}/{}] loss_D: {:.4f} loss_G: {:.4f} time: {:.2f}'.format(
                          epoch+1, batch_idx+1, len(train_loader),
                          d_loss.mean().item(),
                          g_loss.mean().item(),
                          time.time() - batch_time))
        
        # storing generator output after every 10 epochs
        if batch_idx == len(train_loader)-1 and ((epoch+1)%10==0 or epoch==0):
            viz_sample = torch.cat((images[:32], fake_images[:32]), 0)
            vutils.save_image(viz_sample,
            os.path.join(output_save_path, 'output_{}_epoch_{}.png'.format(date,epoch+1)),
                              nrow=8,normalize=True)

# saving the trained models
torch.save(generator.state_dict(), os.path.join(model_save_path, 'generator_{}.pth'.format(date)))
torch.save(discriminator.state_dict(), os.path.join(model_save_path,'discriminator_{}.pth'.format(date)))
        
print('Total train time: {:.2f}'.format(time.time() - start_time))

Epoch: 1
epoch: 1, batch: 1/30
hi after images
hi after wrong images
embeddings size: torch.Size([64, 256])
training the descriminator
torch.Size([3, 256, 1, 1]) torch.Size([64, 256, 1, 1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16384x1 and 256x128)

In [ ]:
# generator loss plot
plt.figure(figsize=(10,5))
plt.title("Generator Loss During Training")
plt.plot(G_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.show()

plt.savefig(os.path.join(output_save_path, 'output_generatorLoss_{}.png'.format(date)))

In [ ]:
# discriminator loss plot
plt.figure(figsize=(10,5))
plt.title("Discriminator Loss During Training")
plt.plot(D_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.show()

plt.savefig(os.path.join(output_save_path, 'output_discriminatorLoss_{}.png'.format(date)))

In [ ]:
# Get all file names from the "generated_images" directory
file_names = os.listdir(output_save_path)
file_names = [name for name in file_names if name.startswith('output_{}_'.format(date))]

# Sort the file names numerically
file_names = sorted(file_names, key=lambda name: int(name.split('_')[3].split('.')[0]))

# Create a list to store the read images
images = []

for file_name in file_names:
    images.append(imageio.imread(os.path.join(output_save_path,file_name)))

imageio.mimsave(os.path.join(output_save_path, 'output_gif_{}.gif'.format(date)), images, fps=1) 

In [ ]:
from IPython.display import Image

# Load the GIF
with open(os.path.join(output_save_path, 'output_gif_{}.gif'.format(date)),'rb') as file:
    display(Image(file.read()))